# 1. Import module

In [1]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd

25168


# 2. Load Data relevant information

In [2]:
analysis_save_folder=r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin\resources'

## 2.1 Get loci info from codebook

In [3]:
# Load sorted codebook (allows direct matrix slicing) with cell type info
# Load codebook (then sort to allow direct matrix slicing) with cell type info

celltype_codebook_fname = os.path.join(analysis_save_folder,'old_merged_codebook_w_celltype.csv')
celltype_codebook_df = pd.read_csv (celltype_codebook_fname, index_col=None)

# sort df temporailiy so matrix can be sliced by df order directly
#celltype_codebook_df = celltype_codebook_df.sort_values(by = ['chr_as_num', 'chr_order'], ignore_index=False)
celltype_codebook_df

name    id  NDB_784  NDB_755  NDB_826  NDB_713  \
0         1:3740000-3760000     1      0.0      0.0      0.0      0.0   
1         1:6240000-6260000     2      0.0      0.0      0.0      0.0   
2         1:8740000-8760000     3      0.0      0.0      0.0      0.0   
3       1:11240000-11260000     4      0.0      0.0      0.0      0.0   
4       1:13740000-13760000     5      0.0      0.0      0.0      0.0   
...                     ...   ...      ...      ...      ...      ...   
1951  X:101910335-102015609  2049      0.0      0.0      0.0      0.0   
1952  X:136772307-136851006  2050      0.0      0.0      0.0      0.0   
1953  X:140481839-140547985  2051      0.0      0.0      0.0      0.0   
1954  X:167122110-167191320  2052      0.0      0.0      0.0      0.0   
1955  X:169962881-170032477  2053      0.0      0.0      0.0      0.0   

      NDB_865  NDB_725  NDB_817  NDB_710  ...  NDB_608  NDB_460  NDB_563  \
0         0.0      0.0      0.0      0.0  ...      0.0      0.0      0.0   
1         0.0      0.0      0.0      0.0  ...      0.0      0.0      0.0   
2         0.0      0.0      0.0      0.0  ...      0.0      0.0      0.0   
3         0.0      0.0      0.0      0.0  ...      0.0      0.0      0.0   
4         0.0      0.0      0.0      0.0  ...      0.0      0.0      0.0   
...       ...      ...      ...      ...  ...      ...      ...      ...   
1951      0.0      0.0      0.0      0.0  ...      0.0      1.0      0.0   
1952      0.0      0.0      0.0      0.0  ...      0.0      0.0      0.0   
1953      0.0      0.0      0.0      0.0  ...      0.0      0.0      0.0   
1954      0.0      0.0      0.0      0.0  ...      0.0      0.0      0.0   
1955      0.0      0.0      0.0      0.0  ...      0.0      0.0      0.0   

      NDB_592  NDB_368  NDB_436  NDB_629  NDB_604  \
0         0.0      0.0      0.0      0.0      0.0   
1         0.0      0.0      0.0      0.0      0.0   
2         0.0      0.0      0.0      0.0      0.0   
3         0.0      0.0      0.0      0.0      0.0   
4         0.0      0.0      0.0      0.0      0.0   
...       ...      ...      ...      ...      ...   
1951      0.0      0.0      0.0      0.0      0.0   
1952      0.0      0.0      0.0      0.0      0.0   
1953      0.0      0.0      0.0      0.0      0.0   
1954      0.0      0.0      0.0      0.0      0.0   
1955      0.0      0.0      0.0      0.0      0.0   

                                              cell_type  chr_as_num  
0                                              ['None']           1  
1                                              ['None']           1  
2                                              ['None']           1  
3                                              ['None']           1  
4                                              ['None']           1  
...                                                 ...         ...  
1951                                          ['L6_CT']          23  
1952                                          ['Oligo']          23  
1953                              ['Astro', 'MicroPVM']          23  
1954                                       ['MicroPVM']          23  
1955  ['Oligo', 'MicroPVM', 'Sncg', 'Lamp5', 'Vip', ...          23  

[1956 rows x 200 columns]

In [4]:
#celltype_codebook_df['library']

In [5]:
#celltype_codebook_df.columns.tolist()

## 2.2 Make a new dataframe to store the transcirption annotation

In [6]:
# Get path for the py containing functions
import os
import sys
import importlib
module_path =r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin'
if module_path not in sys.path:
    sys.path.append(module_path)
    
    # import gene selection modules
import gene_selection
importlib.reload(gene_selection)
import gene_to_loci
importlib.reload(gene_to_loci)

<module 'gene_to_loci' from 'C:\\Users\\shiwei\\Documents\\AnalysisTool_Chromatin\\gene_to_loci.py'>

In [7]:
#loci_df = pd.DataFrame(celltype_codebook_df['name'].tolist(), columns=['name'])
#loci_df = celltype_codebook_df[['name','id','chr','chr_order','library','dtype']]
loci_df = celltype_codebook_df[['name','id','chr','chr_order']]
loci_df

name    id chr  chr_order
0         1:3740000-3760000     1   1          0
1         1:6240000-6260000     2   1          1
2         1:8740000-8760000     3   1          2
3       1:11240000-11260000     4   1          5
4       1:13740000-13760000     5   1          6
...                     ...   ...  ..        ...
1951  X:101910335-102015609  2049   X         35
1952  X:136772307-136851006  2050   X         47
1953  X:140481839-140547985  2051   X         49
1954  X:167122110-167191320  2052   X         60
1955  X:169962881-170032477  2053   X         62

[1956 rows x 4 columns]

In [8]:
# Format the chr loci name and use as index
from gene_to_loci import loci_pos_format
loci_name_list = list(map(loci_pos_format, celltype_codebook_df['name'].tolist()))
loci_name_arr = np.array(loci_name_list)
loci_name_arr 

array([['chr1_3740000_3760000', '1', '3740000', '3760000'],
       ['chr1_6240000_6260000', '1', '6240000', '6260000'],
       ['chr1_8740000_8760000', '1', '8740000', '8760000'],
       ...,
       ['chrX_140481839_140547985', 'X', '140481839', '140547985'],
       ['chrX_167122110_167191320', 'X', '167122110', '167191320'],
       ['chrX_169962881_170032477', 'X', '169962881', '170032477']],
      dtype='<U25')

In [9]:
loci_df['loci_name'] = list(loci_name_arr[:,0])

loci_df['chr_order'] = loci_df['chr_order'].map(lambda x: int(x))

<ipython-input-9-1084f06fc461>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loci_df['loci_name'] = list(loci_name_arr[:,0])
<ipython-input-9-1084f06fc461>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loci_df['chr_order'] = loci_df['chr_order'].map(lambda x: int(x))


In [10]:
loci_df = loci_df.set_index ('loci_name')
loci_df

name    id chr  chr_order
loci_name                                                           
chr1_3740000_3760000          1:3740000-3760000     1   1          0
chr1_6240000_6260000          1:6240000-6260000     2   1          1
chr1_8740000_8760000          1:8740000-8760000     3   1          2
chr1_11240000_11260000      1:11240000-11260000     4   1          5
chr1_13740000_13760000      1:13740000-13760000     5   1          6
...                                         ...   ...  ..        ...
chrX_101910335_102015609  X:101910335-102015609  2049   X         35
chrX_136772307_136851006  X:136772307-136851006  2050   X         47
chrX_140481839_140547985  X:140481839-140547985  2051   X         49
chrX_167122110_167191320  X:167122110-167191320  2052   X         60
chrX_169962881_170032477  X:169962881-170032477  2053   X         62

[1956 rows x 4 columns]

## 2.3 Get all gene annotation from the prepared Ensembl df

In [11]:
# load gene annotation (covering all genes from the SMART-seq) for chr locus
# this should be enough since we can only analyze the genes with info from SMARTseq
# also we used Ensembl for both gene2loci and loci2gene to be consistent 
pool_folder = r'L:\Shiwei\DNA_MERFISH_analysis\SMARTer_nuclei_MOp'
gene_annotation_df = pd.read_csv(os.path.join(pool_folder, "MOp_smart_sn_gene_chr_info_NEW_from_transcriptome_FORMAT.csv"),index_col=0)

gene_annotation_df.head()

chr      start        end                   gene_biotype  \
gene                                                                     
0610005C13Rik   7   45567794   45575327                      antisense   
0610009B22Rik  11   51685386   51688874                 protein_coding   
0610009E02Rik   2   26445696   26459390           processed_transcript   
0610009L18Rik  11  120348678  120351190  bidirectional_promoter_lncRNA   
0610010F05Rik  11   23564961   23633639                 protein_coding   

               coding_strand  length           genomic_position  
gene                                                             
0610005C13Rik             -1    7533     chr7_45567794_45575327  
0610009B22Rik             -1    3488    chr11_51685386_51688874  
0610009E02Rik              1   13694     chr2_26445696_26459390  
0610009L18Rik              1    2512  chr11_120348678_120351190  
0610010F05Rik             -1   68678    chr11_23564961_23633639

In [12]:
from gene_to_loci import find_genes_near_loci

loci_name = loci_df.index[100]
loci_name

sel_genes = find_genes_near_loci (loci_name, 
                          gene_annotation_df,
                          extend_dist = 500*1000, gene_coverage_type = 'tss')

sel_genes

'Gm13582; Psmd14; Rbms1; Tank'

In [13]:
# Function to find genes near a list of loci provided in DataFrame 
from gene_to_loci import find_genes_near_loci

def find_genes_near_loci_for_loci_dataframe (loci_df:pd.core.frame.DataFrame, 
                                             gene_annotation_df:pd.core.frame.DataFrame, 
                                             extend_dist = 50*1000,
                                             gene_coverage_type = 'tss',
                                             key_added = None):


    """Find genes near loci for a list of loci provided in DataFrame"""
    sel_genes_list = []
    for loci_name in loci_df.index.tolist():
        sel_genes =  find_genes_near_loci (loci_name, 
                                           gene_annotation_df,
                                           extend_dist = extend_dist, 
                                           gene_coverage_type = gene_coverage_type
                                          )
        sel_genes_list.append(sel_genes)
    
    if isinstance (key_added, type(None)):
        extend_dist_name = str(int(extend_dist/1000)) + 'kb'
        key_added = f'adjacent_genes_{extend_dist_name}_{gene_coverage_type}'
    
    loci_df[key_added]=sel_genes_list
    
    return loci_df

In [14]:
loci_df_copy = loci_df.copy()

for _dist in [0*1000,10*1000,50*1000,100*1000,200*1000,500*1000,1000*1000,1500*1000,2000*1000,]:
    print(f'Add gene annotation for {_dist}bp.')
    loci_df_copy = find_genes_near_loci_for_loci_dataframe (loci_df_copy, 
                                            gene_annotation_df,
                                             extend_dist = _dist,
                                             gene_coverage_type = 'tss',
                                             key_added = None)

loci_df_copy

Add gene annotation for 0bp.
Add gene annotation for 10000bp.
Add gene annotation for 50000bp.
Add gene annotation for 100000bp.
Add gene annotation for 200000bp.
Add gene annotation for 500000bp.
Add gene annotation for 1000000bp.
Add gene annotation for 1500000bp.
Add gene annotation for 2000000bp.


name    id chr  chr_order  \
loci_name                                                              
chr1_3740000_3760000          1:3740000-3760000     1   1          0   
chr1_6240000_6260000          1:6240000-6260000     2   1          1   
chr1_8740000_8760000          1:8740000-8760000     3   1          2   
chr1_11240000_11260000      1:11240000-11260000     4   1          5   
chr1_13740000_13760000      1:13740000-13760000     5   1          6   
...                                         ...   ...  ..        ...   
chrX_101910335_102015609  X:101910335-102015609  2049   X         35   
chrX_136772307_136851006  X:136772307-136851006  2050   X         47   
chrX_140481839_140547985  X:140481839-140547985  2051   X         49   
chrX_167122110_167191320  X:167122110-167191320  2052   X         60   
chrX_169962881_170032477  X:169962881-170032477  2053   X         62   

                         adjacent_genes_0kb_tss adjacent_genes_10kb_tss  \
loci_name                                                                 
chr1_3740000_3760000                 intergenic              intergenic   
chr1_6240000_6260000                 intergenic              intergenic   
chr1_8740000_8760000                 intergenic              intergenic   
chr1_11240000_11260000               intergenic              intergenic   
chr1_13740000_13760000               intergenic              intergenic   
...                                         ...                     ...   
chrX_101910335_102015609             intergenic              intergenic   
chrX_136772307_136851006            Glra4; Plp1             Glra4; Plp1   
chrX_140481839_140547985             intergenic              intergenic   
chrX_167122110_167191320             intergenic              intergenic   
chrX_169962881_170032477   Erdr1; G530011O06Rik    Erdr1; G530011O06Rik   

                                                 adjacent_genes_50kb_tss  \
loci_name                                                                  
chr1_3740000_3760000                                          intergenic   
chr1_6240000_6260000                               4732440D04Rik; Rb1cc1   
chr1_8740000_8760000                                             Gm15452   
chr1_11240000_11260000                                        intergenic   
chr1_13740000_13760000                                            Gm5523   
...                                                                  ...   
chrX_101910335_102015609                                      intergenic   
chrX_136772307_136851006  BC065397; Glra4; Gm15026; Morf4l2; Plp1; Rab9b   
chrX_140481839_140547985                                           Prps1   
chrX_167122110_167191320                                 Gm15232; Tmsb4x   
chrX_169962881_170032477                            Erdr1; G530011O06Rik   

                                                   adjacent_genes_100kb_tss  \
loci_name                                                                     
chr1_3740000_3760000                                                   Xkr4   
chr1_6240000_6260000                                  4732440D04Rik; Rb1cc1   
chr1_8740000_8760000                                                Gm15452   
chr1_11240000_11260000                                           intergenic   
chr1_13740000_13760000                                       Gm5523; Lactb2   
...                                                                     ...   
chrX_101910335_102015609                                   H3f3a-ps1; Nhsl2   
chrX_136772307_136851006  BC065397; Glra4; Gm15026; Gm6275; Morf4l2; Plp...   
chrX_140481839_140547985                                     Prps1; Tsc22d3   
chrX_167122110_167191320                            Gm15232; Gm8814; Tmsb4x   
chrX_169962881_170032477                               Erdr1; G530011O06Rik   

                                                   adjacent_genes_200kb_tss  \
loci_name                         

In [15]:
# save the file 

loci_df_copy.to_csv(os.path.join(analysis_save_folder,'old_MERFISH_loci_adjacent_genes_tss.csv'))